In [372]:
#imports
try:
    import re
    import json
    import codecs
    from pprint import pprint
    from bs4 import BeautifulSoup
    from urllib.request import quote
    from urllib.request import urlopen
except ImportError:
    print("Error while loading libraries...")
except Exception:
    print('Uncaught exception')
else:
    print("Libraries loaded.")

Libraries loaded.


In [433]:
#constants
mainsite = "http://rozklad.nau.edu.ua/timetable/group/"
NNIIDS = "http://rozklad.nau.edu.ua/timetable/group/%D0%9D%D0%9D%D0%86%D0%86%D0%94%D0%A1"
NNIIDS121 = "http://rozklad.nau.edu.ua/timetable/group/%D0%9D%D0%9D%D0%86%D0%86%D0%94%D0%A1%20121"
institutes=''
lessons = []

In [434]:
soupMain = BeautifulSoup(urlopen(mainsite), 'html.parser')

In [435]:
institutes = extractInstitutes(soupMain)

In [374]:
# html of http://rozklad.nau.edu.ua/timetable/group -> ['ІІКАО', ... , 'ННІІДС', ... , 'ЦНТ н-1']
def extractInstitutes(soupMain):
    return list(map(lambda x: x.text, soupMain.findChildren('select', attrs={'name': 'institute'})))[0].split('\n')[2:-1]

In [404]:
# html of http://rozklad.nau.edu.ua/timetable/group/soupInstitute -> [121, ... , 341, ... , 572]
def extractGroups(soupInstitute):
#     print(soupInstitute)
    return list(soupInstitute.findChildren('select', attrs={'name': 'group'})[0].text.split('\n')[2:-1])
 

In [410]:
# ['ІІКАО', ... , 'ННІІДС', ... , 'ЦНТ н-1'] -> 
#         ['%D0%86%D0%86%D0%9A%D0%90%D0%9E', ... , '%D0%9D%D0%9D%D0%86%D0%86%D0%94%D0%A1', ... , '%D0%A6%D0%9D%D0%A2%20%D0%BD-1']
def encodeNames(lessonNames):
    return list(map(lambda x: quote(x.encode('UTF-8')), lessonNames))


In [411]:
# 'ННІІДС'-> '%D0%9D%D0%9D%D0%86%D0%86%D0%94%D0%A1'
def encodeName(lessonName):
    return quote(lessonName.encode('UTF-8'))


In [377]:
def allWeeks(allScrapedGroups):
    return {k: twoWeeks(v) for (k,v) in allScrapedGroups.items()}

In [378]:
def twoWeeks(soup):
    week1 = soup.findAll('td', attrs={'colspan': '3'})[1:37]
    week2 = soup.findAll('td', attrs={'colspan': '3'})[38:-1]
    return week1+week2

In [379]:
def listLessons(weeks):
    return [makeObj(k,v) for k,v in weeks]

In [482]:
def findTeacher(tag): 
    if not tag.contents: 
        return {}
    index = tag.small.contents[0].find(' ')
    return {
              "degree":"",
              "last_name": tag.small.contents[0][0:index],
              "first_name": tag.small.contents[0][index+1:index+2],
              "middle_name": tag.small.contents[0][index+3:index+4]
           }

In [452]:
#Complete JSON of a lesson
# i: index of a tag: row(day)*6+column(para)
# tag: <td>Глоба...</td>
def makeObj(i,tag):
    if not tag.contents: 
        return {}
    a=0
    if i < 35:
        a = 1
    else: 
        a = 0
    index = tag.small.contents[0].find(' ')
    housing = 'undefined'
    room_num = 'undefined'
    room_floor = 'undefined'
    if not re.findall('\d+',tag.contents[0]):
        housing = ('C/K')
        room_num = ""
        room_floor = ""
    else: 
        housing = re.findall('\d+',tag.contents[0])[0]
        room_num = re.findall('\d+',tag.contents[0])[1]
        room_floor = re.findall('\d+',tag.contents[0])[1][0]
    return {
        'name': tag.b.contents[0], 
        'short_name': tag.b.contents[0], 
        'lesson_time':paraToTime(i%6),
        'day': int(i/6)+1,
        'format': lessonFormat(tag),
        'teachers': [{
                  "degree":"",
                  "last_name": tag.small.contents[0][0:index],
                  "first_name": tag.small.contents[0][index+1:index+2],
                  "middle_name": tag.small.contents[0][index+3:index+4]
               } ], 
        'weeks':a,
        'housing':housing,
        'room_num':room_num,
        'room_floor':room_floor}


In [381]:
# 1. -> 08:00 - 09:35
# 2. -> 09:50 - 11:25
# 3. -> 11:40 - 13:15
# 4. -> 13:30 - 15:05
# 5. -> 15:20 - 16:55
# 6. -> 17:10 - 18:45
# 7. -> 19:00 - 20:35
# 8. -> 20:50 - 22:25

def paraToTime(num):
    schedule = [
        "08:00 - 09:35",
        "09:50 - 11:25",
        "11:40 - 13:15",
        "13:30 - 15:05",
        "15:20 - 16:55",
        "17:10 - 18:45",
        "19:00 - 20:35",
        "20:50 - 22:25"]
    return schedule[num]

In [382]:
#returns lecture or practice
def lessonFormat(tag):
    if 'class' in tag.attrs:
        return 'lecture'
    else:
        return 'practice'

In [383]:
soupMain = BeautifulSoup(urlopen(mainsite), 'html.parser')

soupNNIDS = BeautifulSoup(urlopen(NNIIDS), 'html.parser')

soupNNIDS121 = BeautifulSoup(urlopen(NNIIDS121), 'html.parser')


In [384]:
facultyNames = extractInstitutes(soupMain)

In [385]:
groupNamesEncoded = encodeNames(facultyNames)

In [386]:
print(encodeNames(['ІІКАО'])[0])

%D0%86%D0%86%D0%9A%D0%90%D0%9E


In [439]:
tempUrl = encodeNames(institutes)

In [440]:
print(tempUrl)

['%D0%86%D0%86%D0%9A%D0%90%D0%9E', '%D0%94%D0%9F%20%20%D0%9D%D0%9D%D0%86%D0%9D', '%D0%97%D0%94%D0%9D%20%D0%9D%D0%9D%D0%86%D0%9D', '%D0%9D%D0%9D%D0%86%D0%86%D0%94%D0%A1', '%D0%9D%D0%9D%D0%86%D0%90%D0%9D', '%D0%9D%D0%9D%D0%86%D0%90%D0%9F', '%D0%9D%D0%9D%D0%86%D0%95%D0%91', '%D0%9D%D0%9D%D0%86%D0%95%D0%9C', '%D0%9D%D0%9D%D0%86%D0%9A%D0%86%D0%A2', '%D0%9D%D0%9D%D0%86%D0%9C%D0%92', '%D0%9D%D0%9D%D0%90%D0%9A%D0%86', '%D0%9D%D0%9D%D0%93%D0%9C%D0%86', '%D0%9D%D0%9D%D0%AE%D0%86', '%D0%9F%D0%94%D0%9D%20%D0%9D%D0%9D%D0%86%D0%9D', '%D0%A4%D0%A2%D0%A2', '%D0%A6%D0%9C%D0%9E', '%D0%A6%D0%9D%D0%A2', '%D0%A6%D0%9D%D0%A2%20%D0%BD-1']


In [494]:
data = {}
k={}
for i in range(0,len(tempUrl)):
    print(institutes[i]+'\n')
    tempSoup = BeautifulSoup(urlopen(mainsite+tempUrl[i]), 'html.parser')
    groups = extractGroups(tempSoup)
    if groups:
        for j in groups:
            print(mainsite+encodeName(j))
            x = twoWeeks(soup)
            y = list(filter(lambda x: len(x)>0 , [makeObj(z,x[z]) for z in range(0,71)]))
            teacherList = [dict(t) for t in {tuple(d.items()) for d in list(filter(lambda x: len(x)>0 , [findTeacher(x[z]) for z in range(0,71)]))}]
            
            with open("NAUTest.json", encoding='utf-8') as readData:
                data = json.load(readData)
                data[institutes[i]] = {"lessons": y, "subgroups":"0", "teachers": teacherList}
            with open("NAUTest.json", 'w',encoding='utf-8' ) as out:
                json.dump(data, out, ensure_ascii=False, indent=4)
            y = {}

ІІКАО

ДП  ННІН

ЗДН ННІН

ННІІДС

http://rozklad.nau.edu.ua/timetable/group/%D0%9D%D0%9D%D0%86%D0%86%D0%94%D0%A1%20121
http://rozklad.nau.edu.ua/timetable/group/%D0%9D%D0%9D%D0%86%D0%86%D0%94%D0%A1%20131
http://rozklad.nau.edu.ua/timetable/group/%D0%9D%D0%9D%D0%86%D0%86%D0%94%D0%A1%20132
http://rozklad.nau.edu.ua/timetable/group/%D0%9D%D0%9D%D0%86%D0%86%D0%94%D0%A1%20141
http://rozklad.nau.edu.ua/timetable/group/%D0%9D%D0%9D%D0%86%D0%86%D0%94%D0%A1%20151
http://rozklad.nau.edu.ua/timetable/group/%D0%9D%D0%9D%D0%86%D0%86%D0%94%D0%A1%20162
http://rozklad.nau.edu.ua/timetable/group/%D0%9D%D0%9D%D0%86%D0%86%D0%94%D0%A1%20171
http://rozklad.nau.edu.ua/timetable/group/%D0%9D%D0%9D%D0%86%D0%86%D0%94%D0%A1%20172
http://rozklad.nau.edu.ua/timetable/group/%D0%9D%D0%9D%D0%86%D0%86%D0%94%D0%A1%20211
http://rozklad.nau.edu.ua/timetable/group/%D0%9D%D0%9D%D0%86%D0%86%D0%94%D0%A1%20221
http://rozklad.nau.edu.ua/timetable/group/%D0%9D%D0%9D%D0%86%D0%86%D0%94%D0%A1%20225
http://rozklad.nau.edu.ua/time

http://rozklad.nau.edu.ua/timetable/group/%D0%9D%D0%9D%D0%86%D0%90%D0%95%D0%A2%20412
http://rozklad.nau.edu.ua/timetable/group/%D0%9D%D0%9D%D0%86%D0%90%D0%95%D0%A2%20413
http://rozklad.nau.edu.ua/timetable/group/%D0%9D%D0%9D%D0%86%D0%90%D0%95%D0%A2%20414
http://rozklad.nau.edu.ua/timetable/group/%D0%9D%D0%9D%D0%86%D0%90%D0%95%D0%A2%20415
http://rozklad.nau.edu.ua/timetable/group/%D0%9D%D0%9D%D0%86%D0%90%D0%95%D0%A2%20416
http://rozklad.nau.edu.ua/timetable/group/%D0%9D%D0%9D%D0%86%D0%90%D0%95%D0%A2%20418
http://rozklad.nau.edu.ua/timetable/group/%D0%9D%D0%9D%D0%86%D0%90%D0%95%D0%A2%20501
http://rozklad.nau.edu.ua/timetable/group/%D0%9D%D0%9D%D0%86%D0%90%D0%95%D0%A2%20502
http://rozklad.nau.edu.ua/timetable/group/%D0%9D%D0%9D%D0%86%D0%90%D0%95%D0%A2%20503
http://rozklad.nau.edu.ua/timetable/group/%D0%9D%D0%9D%D0%86%D0%90%D0%95%D0%A2%20504
http://rozklad.nau.edu.ua/timetable/group/%D0%9D%D0%9D%D0%86%D0%90%D0%95%D0%A2%20505
http://rozklad.nau.edu.ua/timetable/group/%D0%9D%D0%9D%D0%86%D0%9

http://rozklad.nau.edu.ua/timetable/group/%D0%9D%D0%9D%D0%86%D0%9A%D0%86%D0%A2%20111
http://rozklad.nau.edu.ua/timetable/group/%D0%9D%D0%9D%D0%86%D0%9A%D0%86%D0%A2%20112
http://rozklad.nau.edu.ua/timetable/group/%D0%9D%D0%9D%D0%86%D0%9A%D0%86%D0%A2%20113
http://rozklad.nau.edu.ua/timetable/group/%D0%9D%D0%9D%D0%86%D0%9A%D0%86%D0%A2%20114
http://rozklad.nau.edu.ua/timetable/group/%D0%9D%D0%9D%D0%86%D0%9A%D0%86%D0%A2%20115
http://rozklad.nau.edu.ua/timetable/group/%D0%9D%D0%9D%D0%86%D0%9A%D0%86%D0%A2%20116
http://rozklad.nau.edu.ua/timetable/group/%D0%9D%D0%9D%D0%86%D0%9A%D0%86%D0%A2%20117
http://rozklad.nau.edu.ua/timetable/group/%D0%9D%D0%9D%D0%86%D0%9A%D0%86%D0%A2%20118
http://rozklad.nau.edu.ua/timetable/group/%D0%9D%D0%9D%D0%86%D0%9A%D0%86%D0%A2%20119
http://rozklad.nau.edu.ua/timetable/group/%D0%9D%D0%9D%D0%86%D0%9A%D0%86%D0%A2%20121
http://rozklad.nau.edu.ua/timetable/group/%D0%9D%D0%9D%D0%86%D0%9A%D0%86%D0%A2%20122
http://rozklad.nau.edu.ua/timetable/group/%D0%9D%D0%9D%D0%86%D0%9

http://rozklad.nau.edu.ua/timetable/group/%D0%9D%D0%9D%D0%90%D0%9AI%20512
http://rozklad.nau.edu.ua/timetable/group/%D0%9D%D0%9D%D0%90%D0%9AI%20513
http://rozklad.nau.edu.ua/timetable/group/%D0%9D%D0%9D%D0%90%D0%9AI%20514
http://rozklad.nau.edu.ua/timetable/group/%D0%9D%D0%9D%D0%90%D0%9AI%20515
http://rozklad.nau.edu.ua/timetable/group/%D0%9D%D0%9D%D0%90%D0%9A%D0%86%20101
http://rozklad.nau.edu.ua/timetable/group/%D0%9D%D0%9D%D0%90%D0%9A%D0%86%20101
http://rozklad.nau.edu.ua/timetable/group/%D0%9D%D0%9D%D0%90%D0%9A%D0%86%20102
http://rozklad.nau.edu.ua/timetable/group/%D0%9D%D0%9D%D0%90%D0%9A%D0%86%20102
http://rozklad.nau.edu.ua/timetable/group/%D0%9D%D0%9D%D0%90%D0%9A%D0%86%20103
http://rozklad.nau.edu.ua/timetable/group/%D0%9D%D0%9D%D0%90%D0%9A%D0%86%20103
http://rozklad.nau.edu.ua/timetable/group/%D0%9D%D0%9D%D0%90%D0%9A%D0%86%20104
http://rozklad.nau.edu.ua/timetable/group/%D0%9D%D0%9D%D0%90%D0%9A%D0%86%20104
http://rozklad.nau.edu.ua/timetable/group/%D0%9D%D0%9D%D0%90%D0%9A%D0%86

http://rozklad.nau.edu.ua/timetable/group/%D0%9D%D0%9D%D0%93%D0%9C%D0%86%20126
http://rozklad.nau.edu.ua/timetable/group/%D0%9D%D0%9D%D0%93%D0%9C%D0%86%20127
http://rozklad.nau.edu.ua/timetable/group/%D0%9D%D0%9D%D0%93%D0%9C%D0%86%20201
http://rozklad.nau.edu.ua/timetable/group/%D0%9D%D0%9D%D0%93%D0%9C%D0%86%20202
http://rozklad.nau.edu.ua/timetable/group/%D0%9D%D0%9D%D0%93%D0%9C%D0%86%20203
http://rozklad.nau.edu.ua/timetable/group/%D0%9D%D0%9D%D0%93%D0%9C%D0%86%20204
http://rozklad.nau.edu.ua/timetable/group/%D0%9D%D0%9D%D0%93%D0%9C%D0%86%20206
http://rozklad.nau.edu.ua/timetable/group/%D0%9D%D0%9D%D0%93%D0%9C%D0%86%20207
http://rozklad.nau.edu.ua/timetable/group/%D0%9D%D0%9D%D0%93%D0%9C%D0%86%20211
http://rozklad.nau.edu.ua/timetable/group/%D0%9D%D0%9D%D0%93%D0%9C%D0%86%20216
http://rozklad.nau.edu.ua/timetable/group/%D0%9D%D0%9D%D0%93%D0%9C%D0%86%20221
http://rozklad.nau.edu.ua/timetable/group/%D0%9D%D0%9D%D0%93%D0%9C%D0%86%20226
http://rozklad.nau.edu.ua/timetable/group/%D0%9D%D0%

In [445]:
x = twoWeeks(soup)

In [450]:
y = list(filter(lambda x: len(x)>0 , [makeObj(z,x[z]) for z in range(0,71)]))


In [451]:
print(y)

[{'name': 'Вища математика', 'short_name': 'Вища математика', 'lesson_time': '08:00 - 09:35', 'day': 1, 'format': 'practice', 'teachers': [{'degree': '', 'last_name': 'Глоба', 'first_name': 'С', 'middle_name': 'О'}], 'weeks': 1, 'housing': '11', 'room_num': '214', 'room_floor': '2'}, {'name': 'Історія та культура України', 'short_name': 'Історія та культура України', 'lesson_time': '09:50 - 11:25', 'day': 1, 'format': 'practice', 'teachers': [{'degree': '', 'last_name': 'Сенчило', 'first_name': 'Н', 'middle_name': 'О'}], 'weeks': 1, 'housing': '11', 'room_num': '214', 'room_floor': '2'}, {'name': 'Вища математика', 'short_name': 'Вища математика', 'lesson_time': '11:40 - 13:15', 'day': 2, 'format': 'lecture', 'teachers': [{'degree': '', 'last_name': 'Глоба', 'first_name': 'С', 'middle_name': 'О'}], 'weeks': 1, 'housing': '11', 'room_num': '311', 'room_floor': '3'}, {'name': 'Інформаційні технології та осн програм', 'short_name': 'Інформаційні технології та осн програм', 'lesson_time': 

In [390]:

k = {"ННІІДС":{"lessons": k, "subgroups":"0", "teachers": []}}
with codecs.open("NAUTest.json", "w", encoding='utf-8') as out:
    json.dump(k, out, ensure_ascii=False, indent=4)

In [390]:
k = [k for k in y if len(k)>0]
k = {"ННІІДС":{"lessons": k, "subgroups":"0", "teachers": []}}
with codecs.open("NAU.json", "w", encoding='utf-8') as out:
    json.dump(k, out, ensure_ascii=False, indent=4)